# Connected and Autonomous Driving - Planning module

----

In this module, we will learn how to navigate a map by exploring some popular pathfinding algorithms. All of the algorithm you will use in this lab have been introduced during the class and you can find more detailed descriptions and representations of these algorithms there.

Do not hesitate to create additional code or markdown cells to answer the questions in this notebook.

In [ ]:
#Import libraries
import math
import queue

## The graph

In order to calculate a best path on a map, we use graph search algorithm. It means we need to look at what our graph is first, and how it represent our map. 

This Graph is simply a set of Vertices and Edges: 
- The Vertices represent the possible points to explore, from which we can start or end. 
- The Edges represent the connections between those Vertices. 


In [ ]:
class Graph():
    
    def __init__(self):
        #The Vertice and edges are stored in the edges dictionnary: 
        #   - every Vertice is stored in the dict.
        #   - every Edge is assigned to the Vertice it starts from
        #   - every Edge is of the form (V,w) where V is the other vertice it leads to, and w the weight of that edge
        self.edges = {}
    
    def add_edges(self, origin, dest, weight=0):
        #Add a new Edge that goes from origin to dest
        #If one of the 2 Vertice does not exist in the dictionary, create it
        if origin not in self.edges:
            self.edges[origin] = []
        if dest not in self.edges:
            self.edges[dest] = []
        self.edges[origin].append((dest, weight))
    
    def neighbors(self, node):
        #Takes a Vertice and returns all other Vertice it has an Edge toward.
        if node not in self.edges:
            print("Vertice not found:",node)
            return[]
        neighbors=[]
        for edge in self.edges[node]:
            neighbors.append(edge[0])
        return(neighbors)
        

    def build_grid(self, G):
        #Takes a 2D list and return and build a rectangular grid in its shape.
        #The values in G provide the weights of the connections leading to that Vertice in the grid
        nb_row, nb_col = len(G), len(G[0])
        for row in range(nb_row):
            for col in range(nb_col):
                for (i,j) in [(-1,0),(1,0),(0,-1),(0,1)]:
                        if (-1<row+i and row+i<nb_row and -1<col+j and col+j<nb_col):
                            self.add_edges((row,col),(row+i,col+j),weight=G[row+i][col+j])

    def cost(self, origin, dest):
        if origin in self.edges and dest in self.edges:
            for edge in self.edges[origin]:
                if edge[0]==dest:
                    return edge[1]
            print("There is no edge between those vertice")
        else:
            print("Vertice not found")
        return[]



g = Graph()
graph = [[0 for column in range(4)]
                      for row in range(4)]
print(graph)
g.build_grid(graph)
print(g.edges)
print(g.neighbors((3,1)))
print(g.cost((0,1),(0,0)))


## Graph search

Let's start with the most intuitive graph search strategy: exploring everything.

### Breadth first search

In this first algorithm, we start from an origin point, and from there explore its neighbors. We repeat the operation for every new neighbors while remembering which Vertice led us to it. Once we reach the destination as one of the new neighbour, we walk back our steps to see all the Vertices on our path.
We need 2 list: one to store the new points to explore at the frontier of our search, and one to store already explored points. In order to get the path to the target, we also need every reached point to 'remember' which Vertice it was reached from.

Fill the missing code below to complete the algorithm:

In [ ]:
def BreadthFirst(graph, start, goal):
   frontier = queue.Queue()
   frontier.put(start )
   came_from = dict() # path A->B is stored as came_from[B] == A
   came_from[start] = None

   while not frontier.empty():
      ##TODO##
      
      
      
      ##TODO##
   return(walkback(came_from, start, goal))


# The walkback function takes the queue of an algorithm with the nodes of the best path
# It returns the list of all nodes in the correct order from start to goal
def walkback(came, start, goal):
   path=[]
   ##TODO##

   

   ##TODO##
   return(path)


print(BreadthFirst(g, (0,0),(2,2)))


- What is the advantage of using a dict() to store the explored Vertices ?
- Why use a queue instead of a list for frontier Vertices ?
- Do you think your algorithm explore many unnecessary Vertices ?
- Are there specific scenario where it can fail to find a best path ?

### Dijkstra

Although the previous method finds a best path, it is not the most efficient. Moreover, it does not work when we are dealing with a weighted graph where some Vertices are more expensive than others. In our application for exemple, not all CARLA waypoints are at the same distance from each other.

To solve this issue, Dijkstra improve Breadth First Search by turning the Queue into a Weighted Queue that look at the cost of visiting new Vertices. Instead of visiting the oldest Vertice in the queue, Dijkstra's visit the lowest total cost note, where the total cost represent the length from that Vertice to the origin.

$$ Vertice_{next} = Frontier [\min(cost)]$$



In [ ]:
def dijkstra(graph, start, goal):
    frontier = queue.PriorityQueue()
    frontier.put(start, 0)
    came_from = dict()
    cost_so_far = dict()
    came_from[start] = None
    cost_so_far[start] = 0

    while not frontier.empty():
        ##TODO##
        
        
                
        ##TODO##
    return(walkback(came_from, start, goal))

- How does the weighted queue improve the previous algorithm ?
- Is your Dijkstra exploring fewer unnecessary Vertices than the previous methodes ? Explain your answer and prove it by showing the number of nodes explored by both methods.
- Modify the grid below to create a scenario where Breadth First Search fails to find a best path and Dijkstra does not. Display the nodes in the found path as well as its total cost for both algorithm.

In [ ]:
g_w = Graph()
graph = [[1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1]]
g_w.build_grid(graph)

dijkstra(g_w, (1,1), (2,6))

## Looking back and ahead: A* algorithm

While Dijkstra's always look at the past from the origin when choosing a new Vertice to explore, A* also look at the destination using a heuristic. This heuristic defines a way to estimate distances in the graph and always estimate the distance between the current Vertice and the destination.

Then, the strategy is the same is Dijkstra's, but adding the heuristic from the current Vertice to the goal to the cost when evaluating the next Vertice to visit:

$$ Vertice_{next} = Frontier [\min(cost + heuristic)]$$

In [ ]:
def heuristic(a, b):
   return abs(a[0] - b[0]) + abs(a[1] - b[1])

def A_star(graph, start, goal):
    frontier = queue.PriorityQueue()
    frontier.put(start, 0)
    came_from = dict()
    cost_so_far = dict()
    came_from[start] = None
    cost_so_far[start] = 0

    while not frontier.empty():
        ##TODO##
        


        ##TODO##
    return(walkback(came_from, start, goal))

- How does the heuristic improve the previous algorithm ? What is the name of the heuristic used above ? Propose another similar heuristic that could be used here, and discuss which one is more adapted.
- Is your A* exploring fewer unnecessary Vertices than previous methodes ? Explain your answer and prove it by showing the number of nodes explored by all methods.

In [ ]:
graph = graph = [[1,1,1,1,10,1,1,1,1],
                 [1,1,1,1,10,1,1,1,1],
                 [1,1,1,1,1,1,1,1,1],
                 [1,1,1,1,10,1,1,1,1],
                 [1,1,1,1,10,1,1,1,1],
                 [1,1,1,1,10,1,1,1,1]]
 
maze = Graph()
maze.build_grid(graph)

start = (0, 0)
end = (0, 5)

path = A_star(maze, start, end)
print(path)


## Moving to CARLA

You have now successfully completed the Jupyter part of this module. In order to integrate this module in the CARLA simulation, go the the global_route_planner.py file. The current planner uses an A* library to calculate a path in the _path_search() method. 

Uncomment the astar_path() method above it and finish its code using what you learned here. Then replace the currently called nx.astar() in _path_search() with self.astar_path.

----

You can run the Simulation_control.py from the previous module again to verify that your car takes the same route to reach its destination in each scenario. 
Run it without the --scenario argument as well to let the car choose random destinations and observe if the path is correct.

You may also use the --loop argument to let the car find a new random destination after it has reached its goal.